In [1]:
# 1.1A-

import numpy as np

# Load the 'images.npy' file with allow_pickle=True
images_data = np.load(r"C:\Users\user\Downloads\Images.npy", allow_pickle=True)

# Print the contents of the 'images.npy' file
print(images_data)



MemoryError: 

In [ ]:
# Print the first few elements to understand their structure
for i in range(5):
    print("Element", i, ":", images_data[i])


In [ ]:
# 1.1B-

import numpy as np
from PIL import Image, ImageDraw

# Step 1: Separate image arrays and annotations
image_arrays = [element[0] for element in images_data]
annotations = [element[1] for element in images_data]

# Step 2: Resize all images to a unified shape (assuming desired_shape is (224, 224))
desired_shape = (224, 224)

resized_images = []
for image in image_arrays:
    img = Image.fromarray(image)
    img = img.resize(desired_shape, Image.ANTIALIAS)  # Resize while preserving aspect ratio
    resized_images.append(np.array(img))

# Step 3: Create binary masks for each image, replacing pixels within the masked area with 1
masked_images = []
for img, annotation in zip(resized_images, annotations):
    mask = Image.new('L', img.shape[:2], 0)  # Create a black mask
    draw = ImageDraw.Draw(mask)
    for obj in annotation:
        if 'Face' in obj['label']:
            points = [(point['x'] * img.shape[1], point['y'] * img.shape[0]) for point in obj['points']]
            draw.polygon(points, fill=1)  # Fill the polygon representing the face with 1 in the mask
    masked_images.append(np.array(mask))

# Step 4: Extract features (X) and labels (Y)
X = np.array(masked_images)
Y = np.array(masked_images)  # Use the same mask as labels

# Check shapes
print("Shape of X (features):", X.shape)
print("Shape of Y (labels):", Y.shape)


In [ ]:
# 1.1C-

from sklearn.model_selection import train_test_split

# Assuming X and Y are the features and labels arrays created earlier

# Split the data into 70% train and 30% test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Print the shapes of the resulting datasets
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of Y_train:", Y_train.shape)
print("Shape of Y_test:", Y_test.shape)


In [ ]:
# 1.2A-

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Normalize X
X = X / 255.0

# Ensure Y is binary
Y = (Y > 0).astype(np.float32)

# Add channel dimension to X to match input shape requirements of the model
X = np.expand_dims(X, axis=-1) if X.ndim == 3 else X
Y = np.expand_dims(Y, axis=-1) if Y.ndim == 3 else Y

# Further split the training data into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

# Step 2: Model Design
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(1, (1, 1), activation='sigmoid', padding='same')
])

# Step 3: Model Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Model Training
history = model.fit(
    X_train, Y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, Y_val)
)

# Step 5: Model Evaluation
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()



In [ ]:
# 1.2A

import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Conv2D, UpSampling2D, Input
from tensorflow.keras.models import Model

# Define the input shape
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)

# Load the MobileNet model with pre-trained weights
base_model = MobileNet(input_shape=input_shape, include_top=False, weights='imagenet')

# Freeze the MobileNet layers
base_model.trainable = False

# Encoder: Get the output of the base model
encoder_output = base_model.output  # 7x7

# Decoder: Upsampling path without concatenation
up1 = UpSampling2D(size=(2, 2))(encoder_output)
up1 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)
up1 = Conv2D(256, (3, 3), activation='relu', padding='same')(up1)  # 14x14

up2 = UpSampling2D(size=(2, 2))(up1)
up2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)
up2 = Conv2D(128, (3, 3), activation='relu', padding='same')(up2)  # 28x28

up3 = UpSampling2D(size=(2, 2))(up2)
up3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)
up3 = Conv2D(64, (3, 3), activation='relu', padding='same')(up3)  # 56x56

up4 = UpSampling2D(size=(2, 2))(up3)
up4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up4)
up4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up4)  # 112x112

up5 = UpSampling2D(size=(2, 2))(up4)
up5 = Conv2D(16, (3, 3), activation='relu', padding='same')(up5)
up5 = Conv2D(16, (3, 3), activation='relu', padding='same')(up5)  # 224x224

# Output layer
outputs = Conv2D(1, (1, 1), activation='sigmoid')(up5)

# Define the model
model = Model(inputs=[inputs], outputs=[outputs])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()



In [ ]:
# 1.2B-

def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2.0 * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)




In [ ]:
# 1.2C-

import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, UpSampling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt

# Normalize X
X = X / 255.0

# Ensure Y is binary
Y = (Y > 0).astype(np.float32)

# Add channel dimension to X to match input shape requirements of the model
X = np.expand_dims(X, axis=-1) if X.ndim == 3 else X
Y = np.expand_dims(Y, axis=-1) if Y.ndim == 3 else Y

# Further split the training data into train and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)

# Step 2: Model Design
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    UpSampling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu', padding='same'),
    Conv2D(1, (1, 1), activation='sigmoid', padding='same')
])

# Step 3: Model Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Step 4: Define Callbacks
checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

callbacks = [checkpoint, early_stopping, reduce_lr]

# Step 5: Train the Model
history = model.fit(
    X_train, Y_train,
    epochs=20,
    batch_size=32,
    validation_data=(X_val, Y_val),
    callbacks=callbacks
)

# Step 6: Evaluate the Model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.show()


Model 1 (Epochs: 10)

    Training Loss: Started with a high loss of around 3.83 and gradually decreased to about 1.99.
    Training Accuracy: Improved from approximately 63% to 87%.
    Validation Loss: Initially started at 2.237 and remained constant throughout training.
    Validation Accuracy: Stayed at around 86% throughout training.
    Test Loss: 2.122
    Test Accuracy: 86.72%
    
    
Model 2 (Epochs: 20)

    Training Loss: Started at 0.617 and decreased to around 0.35.
    Training Accuracy: Improved from approximately 85% to 87.78%.
    Validation Loss: Decreased from 0.4278 to 0.3776.
    Validation Accuracy: Stayed around 85.96%.
    Test Loss: 2.122
    Test Accuracy: 86.72%
    
    
Insights:

    Both models achieved similar test accuracy, suggesting that they have comparable performance on unseen data.
    Model 2 showed better convergence in terms of training and validation loss, indicating better generalization capability.
    However, neither model showed significant improvement in validation accuracy after a certain number of epochs, suggesting that further training might not yield much improvement.
    It seems like the models might have reached their limits in terms of learning from the provided data.

In [ ]:
import matplotlib.pyplot as plt

# Load the test image with index 3 from the test data
test_image_index = 3
test_image = X_test[test_image_index]  # Assuming X_test contains the test images
test_mask = Y_test[test_image_index]  # Assuming Y_test contains the corresponding masks

# Visualize the test image and its mask
plt.figure(figsize=(10, 5))

# Original test image
plt.subplot(1, 2, 1)
plt.imshow(test_image)
plt.title("Test Image")
plt.axis('off')

# Masked image overlay
plt.subplot(1, 2, 2)
plt.imshow(test_image)
plt.imshow(test_mask, cmap='jet', alpha=0.5)  # Overlay the mask with a color map and some transparency
plt.title("Masked Image")
plt.axis('off')

plt.show()


In [ ]:
# 2.1A--

import cv2
import os

# Function to read images from folder
def read_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.imread(os.path.join(folder_path, filename))
        if img is not None:
            images.append(img)
        else:
            print("Failed to read image:", filename)
    return images

# Path to the folder containing images
folder_path = r"C:\Users\user\Downloads\training_images-20211126T092819Z-001\training_images"

# Read images from folder
images = read_images_from_folder(folder_path)

# Check the number of images read
print("Number of images read:", len(images))


In [ ]:
# 2.1B--

import cv2
import os

# Function to detect faces in an image
def detect_faces(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Path to the folder containing images
folder_path = r"C:\Users\user\Downloads\training_images-20211126T092819Z-001\training_images"
# Output folder for images with detected faces
output_folder = "images_with_faces"

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    # Read the image
    img_path = os.path.join(folder_path, filename)
    img = cv2.imread(img_path)
    
    # Check if image was loaded successfully
    if img is None:
        print("Failed to load image:", img_path)
        continue
    
    # Detect faces in the image
    faces = detect_faces(img)
    
    # Check if faces were detected
    if len(faces) == 0:
        print("No faces detected in image:", img_path)
        continue
    
    # Draw rectangles around the detected faces
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
    
    # Save the image with detected faces
    output_path = os.path.join(output_folder, filename)
    cv2.imwrite(output_path, img)
    print("Image with detected faces saved:", output_path)

print("Face detection completed.")


In [ ]:
# 2.1C--

import cv2
import os
import pandas as pd

# Function to detect faces and extract metadata
def detect_faces_and_metadata(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    metadata = []
    for (x, y, w, h) in faces:
        metadata.append({'x': x, 'y': y, 'width': w, 'height': h})
    return metadata

# Path to the folder containing images
folder_path = r"C:\Users\user\Downloads\training_images-20211126T092819Z-001\training_images"

# DataFrame to store face metadata
face_metadata_df = pd.DataFrame(columns=['Image', 'FaceMetadata'])

# Loop through all images in the folder
for filename in os.listdir(folder_path):
    # Read the image
    img_path = os.path.join(folder_path, filename)
    img = cv2.imread(img_path)
    
    # Check if image was loaded successfully
    if img is None:
        print("Failed to load image:", img_path)
        continue
    
    # Detect faces and extract metadata
    face_metadata = detect_faces_and_metadata(img)
    
    # Append face metadata to DataFrame
    face_metadata_df = face_metadata_df.append({'Image': filename, 'FaceMetadata': face_metadata}, ignore_index=True)

# Display DataFrame with face metadata
print("DataFrame with face metadata:")
print(face_metadata_df)


In [ ]:
# 2.1D--

# Define the path where you want to save the CSV file
csv_file_path = "face_metadata.csv"

# Save the DataFrame to a CSV file
face_metadata_df.to_csv(csv_file_path, index=False)

print("DataFrame saved to CSV file:", csv_file_path)


In [ ]:
# Part1.1d--

import numpy as np
import matplotlib.pyplot as plt
import cv2

# Load the images
images_path = r"C:\Users\user\Downloads\Images.npy"  # Replace this with the actual path to your 'Images.npy' file
images_data = np.load(images_path, allow_pickle=True)

# Generate masked images
masked_images = []
for image_info in images_data:
    image_array = image_info[0]
    annotations = image_info[1]

    # Convert to grayscale if necessary
    if len(image_array.shape) > 2:
        image_array = np.mean(image_array, axis=2).astype(np.uint8)

    # Create a mask with the same dimensions as the image, filled with zeros
    mask = np.zeros_like(image_array, dtype=np.uint8)

    # Draw polygons for each face annotation
    for annotation in annotations:
        if 'Face' in annotation['label']:  # Assuming 'Face' label indicates face annotations
            points = [(point['x'] * image_array.shape[1], point['y'] * image_array.shape[0]) for point in annotation['points']]
            # Convert polygon points to integer and reshape for fillPoly
            points = np.array(points, np.int32).reshape((-1, 1, 2))
            cv2.fillPoly(mask, [points], color=255)

    # Apply the mask to the original image
    masked_image = cv2.bitwise_and(image_array, image_array, mask=mask)

    masked_images.append(masked_image)

# Visualize some images and masks
num_samples_to_visualize = 5
for i in range(num_samples_to_visualize):
    plt.subplot(2, num_samples_to_visualize, i + 1)
    plt.imshow(images_data[i][0], cmap='gray')
    plt.title("Image {}".format(i + 1))
    plt.axis('off')

    plt.subplot(2, num_samples_to_visualize, num_samples_to_visualize + i + 1)
    plt.imshow(masked_images[i], cmap='gray')
    plt.title("Masked Image {}".format(i + 1))
    plt.axis('off')

plt.show()